# Neural Language Model for Text Generation

## Data Preparation

In [1]:
import string
import re

In [2]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [3]:
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [4]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [5]:
# load document
in_filename = 'republic_clean.txt'
doc = load_doc(in_filename)
print(doc[:200])

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in what


In [6]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid',

In [7]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 118633


In [9]:
sequences[0]

'book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was'

In [10]:
sequences[1]

'i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted'

In [11]:
sequences[2]

'i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was delighted with'

In [12]:
# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

## Train Language Model

In [13]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [14]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [15]:
# define the model
def define_model(vocab_size, seq_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 50, input_length=seq_length))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    # compile network
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize defined model
    model.summary()
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [16]:
# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [17]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [18]:
print(lines[0])
print(sequences[0])

book i i went down yesterday to the piraeus with glaucon the son of ariston that i might offer up my prayers to the goddess bendis the thracian artemis and also because i wanted to see in what manner they would celebrate the festival which was a new thing i was
[1046, 11, 11, 1045, 329, 7409, 4, 1, 2873, 35, 213, 1, 261, 3, 2251, 9, 11, 179, 817, 123, 92, 2872, 4, 1, 2249, 7408, 1, 7407, 7406, 2, 75, 120, 11, 1266, 4, 110, 6, 30, 168, 16, 49, 7405, 1, 1609, 13, 57, 8, 549, 151, 11, 57]


In [20]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

7410


In [21]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [22]:
X[0]

array([1046,   11,   11, 1045,  329, 7409,    4,    1, 2873,   35,  213,
          1,  261,    3, 2251,    9,   11,  179,  817,  123,   92, 2872,
          4,    1, 2249, 7408,    1, 7407, 7406,    2,   75,  120,   11,
       1266,    4,  110,    6,   30,  168,   16,   49, 7405,    1, 1609,
         13,   57,    8,  549,  151,   11])

In [26]:
print(X.shape)

(118633, 50)


In [27]:
# define model
model = define_model(vocab_size, seq_length)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            370500    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 7410)              748410    
Total params: 1,269,810
Trainable params: 1,269,810
Non-trainable params: 0
_________________________________________________________________


In [28]:
# fit model
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
927/927 [==============================] - 189s 204ms/step - loss: 6.1386 - accuracy: 0.0744
Epoch 2/100
927/927 [==============================] - 199s 215ms/step - loss: 5.6793 - accuracy: 0.1066
Epoch 3/100
927/927 [==============================] - 173s 187ms/step - loss: 5.4510 - accuracy: 0.1304
Epoch 4/100
927/927 [==============================] - 185s 200ms/step - loss: 5.2938 - accuracy: 0.1440
Epoch 5/100
927/927 [==============================] - 195s 210ms/step - loss: 5.1797 - accuracy: 0.1520
Epoch 6/100
927/927 [==============================] - 197s 213ms/step - loss: 5.0864 - accuracy: 0.1587
Epoch 7/100
927/927 [==============================] - 186s 201ms/step - loss: 5.0069 - accuracy: 0.1637
Epoch 8/100
927/927 [==============================] - 188s 202ms/step - loss: 4.9346 - accuracy: 0.1674
Epoch 9/100
927/927 [==============================] - 184s 199ms/step - loss: 4.8652 - accuracy: 0.1712
Epoch 10/100
927/927 [==============================] -

In [29]:
# save the model to file
model.save('nl_Text_genertion_model.h5')
# save the tokenizer
dump(tokenizer, open('nl_Text_genertion_tokenizer.pkl', 'wb'))

## Generate Sequence

In [1]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [8]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close
    return text

In [9]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [10]:
# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

In [11]:
# load the model
model = load_model('nl_Text_genertion_model.h5')

In [12]:
# load the tokenizer
tokenizer = load(open('nl_Text_genertion_tokenizer.pkl', 'rb'))

In [13]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

carried on by them how why of course they will go on expeditions together and will take with them any of their children who are strong enough that after the manner of the artisans child they may look on at the work which they will have to do when they are



In [14]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
grown up at the same time highspirited and magnanimous are not to be ascertained the shore i said to the state of epeus the soul of aeschylus will be the best of the soul scaring the other to the hatred of his which we call the shafts of the soul
